# mdd

> Objects representing a multivalued decision diagram.

In [ ]:
#| default_exp mdd

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from collections.abc import Hashable
import dataclasses

In [ ]:
#| export
@dataclasses.dataclass(frozen=True)
class MDDNode:
    """MDDNode represents a single node in the MDD.

    An MDDNode is uniquely identified by its layer and state.  The (node) state must be a hashable object.

    Parameters
    ----------
    layer : int
        layer the node is in
    state : Hashable
        state associated with node

    """
    layer: int
    state: Hashable

    def __str__(self) -> str:
        return f"N_{self.layer}({self.state})"

`MDDNode` represents a node in the decision diagram. It is uniquely identified by which layer the node is located in, and the node's state. The state should be hashable to ensure we can quickly identify when two nodes in the same layer are "equivalent".

In [ ]:
n1 = MDDNode(0, (1, 2, 3))
n2 = MDDNode(1, (1, 3))
print(f"n1 = {n1}\nn2 = {n2}")

n1 = N_0((1, 2, 3))
n2 = N_1((1, 3))


In [ ]:
#| export
@dataclasses.dataclass()
class MDDArc:
    """MDDArc represents a single arc in the MDD.

    An MDDArc is uniquely identified by its head/tail nodes, label, and weight.

    Parameters
    ----------
    label : Hashable
        label of arc (e.g., assigned value)
    weight : float
        weight of arc (e.g., coefficient)
    tail : MDDNode
        tail/source node
    head : MDDNode
        head/destination node

    """
    label: Hashable
    weight: float
    tail: MDDNode
    head: MDDNode

    def __str__(self) -> str:
        return f"A({self.label},{self.weight}:{self.tail},{self.head})"

`MDDArc` represents an arc in the decision diagram. It is uniquely identified by the combaintion of its tail node, head node, and label.

Unlike `MDDNode`, `MDDArc` is NOT immutable; in particular, `MDDArc.weight` may be changed later.

In [ ]:
a = MDDArc(2, 0, n1, n2)
print(f"Original: {a}")
a.weight = 2
print(f"After modification: {a}")

Original: A(2,0:N_0((1, 2, 3)),N_1((1, 3)))
After modification: A(2,2:N_0((1, 2, 3)),N_1((1, 3)))


In [ ]:
#| export
@dataclasses.dataclass()
class MDDNodeInfo:
    """MDDNodeInfo represents information associated with an MDDNode.

    Parameters
    ----------
    incoming : list[MDDArc]
        list of incoming arcs (default: [])
    outgoing : list[MDDArc]
        list of outgoing arcs (default: [])

    """
    incoming: list[MDDArc] = dataclasses.field(default_factory=list)
    outgoing: list[MDDArc] = dataclasses.field(default_factory=list)

    def __str__(self) -> str:
        incoming_str = ", ".join(str(a) for a in self.incoming)
        outgoing_str = ", ".join(str(a) for a in self.outgoing)
        return f"<in=[{incoming_str}], out=[{outgoing_str}]>"

`MDDNodeInfo` represents information assicated with a node in the decision diagram, and in particular its incoming and outgoing arcs.

In [ ]:
info = MDDNodeInfo(incoming=[], outgoing=[a])
print(f"info = {info}")

info = <in=[], out=[A(2,2:N_0((1, 2, 3)),N_1((1, 3)))]>


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()